# Simulating a Resonant Cavity with Meep

## Introduction to Meep and FDTD

[Meep](https://meep.readthedocs.io/en/latest/) is a free and open-source software package for electromagnetic simulations using the finite-difference time-domain (FDTD) method. FDTD is a popular method for solving Maxwell's equations, which describe how electromagnetic waves propagate and interact with materials.

In this notebook, we will use Meep to simulate a resonant cavity, which is a structure that can trap electromagnetic waves at specific frequencies, called resonant frequencies. We will start with a simple 1D cavity and then move to a more complex 2D cavity.

In [ ]:
import meep as mp
import numpy as np
import matplotlib.pyplot as plt

## Simulating a 1D Resonant Cavity

Let's start with a simple 1D resonant cavity. The cavity is formed by two mirrors (perfect electric conductors) separated by a distance. We will place a source inside the cavity to excite the modes and then observe how the fields evolve in time.

In [ ]:
cell_size = mp.Vector3(z=14)
geometry = [mp.Block(center=mp.Vector3(z=-7), size=mp.Vector3(mp.inf, mp.inf, 0), material=mp.metal),
            mp.Block(center=mp.Vector3(z=7), size=mp.Vector3(mp.inf, mp.inf, 0), material=mp.metal)]

sources = [mp.Source(src=mp.GaussianSource(frequency=0.15, fwidth=0.1), 
                    component=mp.Ey, 
                    center=mp.Vector3(z=0))]

pml_layers = [mp.PML(1.0)]

resolution = 10

Now, we create the `Simulation` object, which is the main object in Meep that brings together the geometry, sources, resolution, and other simulation parameters.

In [ ]:
sim = mp.Simulation(cell_size=cell_size,
                    boundary_layers=pml_layers,
                    geometry=geometry,
                    sources=sources,
                    resolution=resolution)

Finally, we run the simulation. We will run it for 200 time steps and visualize the electric field (Ey component) at each time step. The `mp.at_every` function allows us to specify a function to be called at certain intervals during the simulation.

In [ ]:
plt.figure(dpi=100)
sim.run(mp.at_every(2, mp.plot2D(fields=mp.Ey, output_plane=mp.Volume(center=mp.Vector3(), size=mp.Vector3(z=14)))), until=200)
plt.show()

## Finding the Resonant Modes

To find the resonant modes of the cavity, we can use a tool called [Harminv](https://meep.readthedocs.io/en/latest/Python_User_Interface/#harminv). Harminv is a tool that can extract the frequencies and decay rates of the modes from the time-domain simulation data.

We will run the simulation again, but this time we will use `Harminv` to analyze the fields at a specific point in the cavity. We will then print the results.

In [ ]:
sim.reset_meep()

h = mp.Harminv(mp.Ey, mp.Vector3(z=0), 0.1, 0.2)

sim.run(after_sources=h, until_after_sources=200)

print(h.modes)

## Extending to a 2D Resonant Cavity

Now, let's extend our simulation to 2D. We will create a 2D resonant cavity by using a `Cylinder` object as our cavity. The setup is similar to the 1D case, but we now have a 2D geometry and we will visualize the fields in 2D.

In [ ]:
cell_size = mp.Vector3(14, 14)
geometry = [mp.Cylinder(radius=5, material=mp.metal)]

sources = [mp.Source(src=mp.GaussianSource(frequency=0.15, fwidth=0.1), 
                    component=mp.Ez, 
                    center=mp.Vector3(0,0))]

pml_layers = [mp.PML(1.0)]

resolution = 10

In [ ]:
sim = mp.Simulation(cell_size=cell_size,
                    boundary_layers=pml_layers,
                    geometry=geometry,
                    sources=sources,
                    resolution=resolution)

In [ ]:
plt.figure(dpi=100)
sim.run(mp.at_every(0.6, mp.plot2D(fields=mp.Ez)), until=200)
plt.show()

## Conclusion and Further Exploration

In this notebook, we have learned how to simulate a resonant cavity in 1D and 2D using Meep. We have also seen how to use Harminv to find the resonant modes of the cavity.

For further exploration, you can try:
*   Changing the geometry of the cavity (e.g., using different shapes or materials).
*   Using different sources to excite the modes.
*   Exploring other analysis tools in Meep, such as flux monitors to calculate the transmission and reflection spectra.
*   Extending the simulation to 3D.